In [ ]:
from google.cloud import bigquery

# Initialize the BigQuery client
client = bigquery.Client.from_service_account_json('/Users/servandodavidtorresgarcia/servando/relu/gcp/local-catalyst-398309-52645c242657.json')


# Define your table and destination URI
dataset_name = 'thelook_ecommerce'
table_name = 'distribution_centers'
bucket_name = 'ecommerce_looker_download'
destination_uri = f"gs://{bucket_name}/{table_name}.csv"

# Create an extract job
dataset_ref = client.dataset(dataset_name, project='bigquery-public-data')
table_ref = dataset_ref.table(table_name)

extract_job = client.extract_table(
    table_ref,
    destination_uri,
    location='US',  # Location must match that of the source table.
)
extract_job.result()  # Wait for job to complete

print(f"Exported {dataset_name}.{table_name} to {destination_uri}")


In [1]:

from src.data.dataset_reader import DatasetReader
from pathlib import Path
import pandas as pd
pd.set_option('display.max_columns', None)
%load_ext autoreload
%autoreload 2
# ignore warnings
import warnings
warnings.filterwarnings('ignore')
data_folder = Path("../data/raw").resolve()

products = DatasetReader(data_folder).get_data_csv('products.csv')
orders = DatasetReader(data_folder).get_data_csv('orders.csv')
order_items = DatasetReader(data_folder).get_data_csv('order_items.csv')
inventory_items = DatasetReader(data_folder).get_data_csv('inventory_items.csv')

In [ ]:
order_items.query('id == 27569')

In [ ]:
orders.query("order_id == 18975")

In [ ]:
products.query("id == 19220")

In [ ]:
inventory_items.query("id == 74342")

In [2]:
# Select specific columns from each DataFrame
order_items_selected = order_items[['product_id','order_id','status', 'created_at', 'shipped_at', 'delivered_at', 'returned_at', 'sale_price']]
orders_selected = orders[['order_id', 'gender', 'num_of_item']]  # Including 'order_id' for joining
products_selected = products[['id', 'cost', 'category', 'name', 'brand', 'retail_price', 'department', 'sku']]  # Including 'id' for joining

# Perform the join operations based on the relationships identified in the queries
# Assuming 'order_id' is common between 'order_items' and 'orders'
# Assuming 'product_id' is common between 'order_items' and 'products'

merged_df = order_items_selected \
    .merge(orders_selected, on='order_id', how='inner') \
    .merge(products_selected, left_on='product_id', right_on='id', how='inner')



In [3]:
merged_df.drop(columns=['product_id', 'order_id', 'id'], inplace=True)

In [4]:
# place sku to the beginning of the dataframe
cols = list(merged_df.columns)
cols.insert(0, cols.pop(cols.index('sku')))
merged_df = merged_df.loc[:, cols]


In [5]:
merged_df.head()
# checl if sale_price is equal to retail_price
merged_df.query('sale_price != retail_price')
merged_df.drop(columns=['retail_price'], inplace=True)

In [6]:
print(merged_df.head().to_string(index=False))

                             sku   status                     created_at                     shipped_at                   delivered_at returned_at  sale_price gender  num_of_item    cost    category                                    name            brand department
4ACBEDBE977480D19B7B682D4878CAE2  Shipped        2023-08-20 21:21:15 UTC        2023-08-21 07:58:00 UTC                            NaN         NaN         2.5      F            4  1.1075 Accessories Elegant PASHMINA SCARF WRAP SHAWL STOLE Scarf_tradinginc      Women
4ACBEDBE977480D19B7B682D4878CAE2 Complete 2023-09-08 00:38:23.869168 UTC 2023-09-07 19:56:53.869168 UTC 2023-09-08 14:07:53.869168 UTC         NaN         2.5      F            4  1.1075 Accessories Elegant PASHMINA SCARF WRAP SHAWL STOLE Scarf_tradinginc      Women
E0F19F64F086E393CEB0CF4A8C561B51  Shipped        2023-08-24 21:44:30 UTC        2023-08-21 07:58:00 UTC                            NaN         NaN        64.0      F            4 30.2080      Shorts 

In [7]:
merged_df.query('sku == "4ACBEDBE977480D19B7B682D4878CAE2"')

,sku,status,created_at,shipped_at,delivered_at,returned_at,sale_price,gender,num_of_item,cost,category,name,brand,department
0,4ACBEDBE977480D19B7B682D4878CAE2,Shipped,2023-08-20 21:21:15 UTC,2023-08-21 07:58:00 UTC,NaN,NaN,2.5,F,4,1.1075,Accessories,Elegant PASHMINA SCARF WRAP SHAWL STOLE,Scarf_tradinginc,Women
1,4ACBEDBE977480D19B7B682D4878CAE2,Complete,2023-09-08 00:38:23.869168 UTC,2023-09-07 19:56:53.869168 UTC,2023-09-08 14:07:53.869168 UTC,NaN,2.5,F,4,1.1075,Accessories,Elegant PASHMINA SCARF WRAP SHAWL STOLE,Scarf_tradinginc,Women


In [8]:
merged_df.status.unique()

array(['Shipped', 'Complete', 'Processing', 'Returned', 'Cancelled'],
      dtype=object)

In [9]:
#trainsform dates to datetime
merged_df['created_at'] = pd.to_datetime(merged_df['created_at'])
merged_df['shipped_at'] = pd.to_datetime(merged_df['shipped_at'])
merged_df['delivered_at'] = pd.to_datetime(merged_df['delivered_at'])
merged_df['returned_at'] = pd.to_datetime(merged_df['returned_at'])

In [10]:
merged_df.head()
merged_df.groupby('sku').size().reset_index(name='sales')
# create column sales that is the number sales per sku
# Correct way to calculate sales per SKU
merged_df['sales'] = merged_df.groupby('sku')['num_of_item'].transform('sum')


In [ ]:
merged_df.head(3
               )

In [ ]:
merged_df.query('sku == "4ACBEDBE977480D19B7B682D4878CAE2"')
# TODO see duplicated skus with their corresponding sale and decide what to do with them

In [11]:
from src.data.eda import Eda
eda = Eda()
eda.missing_values_table(merged_df)

The selected dataframe has 15 columns and 5 columns with missing values.


,Missing Values,% of Total Values
returned_at,162234,89.8
delivered_at,117399,65.0
shipped_at,63212,35.0
brand,142,0.1
name,12,0.0


In [ ]:
# inspecting for seasonality
from src.data.seasonality import SeasonalityInspector, QuickSeasonalityInspector


# Initialize
inspector = SeasonalityInspector()
quick_inspector = QuickSeasonalityInspector()

# Load your data into a DataFrame called 'data'
# data = pd.read_csv('your_data.csv')

# Plot time series
#inspector.plot_time_series(merged_df, 'created_at', 'sales')



In [ ]:
import numpy as np
# Find dominant period through FFT
dominant_period = quick_inspector.fast_fourier_transform(merged_df, 'sales')
print(f"Dominant period: {dominant_period}")


# Initialize default N
N = 7

# Check if dominant_period is finite
if np.isfinite(dominant_period):
    # Run rolling stats only if dominant_period is finite
    is_mean_stable, is_var_stable = quick_inspector.rolling_stats(merged_df, 'sales', window=int(dominant_period))
    
    # Update N based on rolling stats
    if not (is_mean_stable and is_var_stable):
        N = int(dominant_period)

print(f"Optimal N: {N}")


In [12]:
from src.data.feature_engineering import FeatureEngineeringProcess
fep = FeatureEngineeringProcess()


In [13]:
merged_df_fe1 = fep.price_sales_correlation_features_updated(merged_df, 7, [(-5.0, 1.0), (-3.0, 1.0), (-2.0, 1.0), (1.0, 1.0), (-1.0, 0.5), (-1.0, 0.33)],
                                                             'sku', 'created_at', 'sale_price', 'sales')

In [14]:
from src.data.eda import Eda
eda = Eda()

In [15]:
eda.missing_values_table(merged_df_fe1)

The selected dataframe has 21 columns and 11 columns with missing values.


,Missing Values,% of Total Values
returned_at,162234,89.8
f_corr_-5.0_1.0,148221,82.1
f_corr_-3.0_1.0,148221,82.1
f_corr_-2.0_1.0,148221,82.1
f_corr_1.0_1.0,148221,82.1
f_corr_-1.0_0.5,148221,82.1
f_corr_-1.0_0.33,148221,82.1
delivered_at,117399,65.0
shipped_at,63212,35.0
brand,142,0.1


In [16]:
# handle missing values in price_sales_correlation 
cols_to_fill_zero = [col for col in merged_df_fe1.columns if 'f_corr_' in col]
merged_df_fe1[cols_to_fill_zero] = merged_df_fe1[cols_to_fill_zero].fillna(0)


In [17]:
# Drop uneccesary date columns 
merged_df_fe1.drop(columns=['shipped_at', 'delivered_at', 'returned_at'], inplace=True)

In [18]:
eda.missing_values_table(merged_df_fe1)

The selected dataframe has 18 columns and 2 columns with missing values.


,Missing Values,% of Total Values
brand,142,0.1
name,12,0.0


In [19]:
merged_df_fe2 =  fep.normalize_features(merged_df_fe1, [5, 10, 15], 28, 'sku', 'created_at', 'sale_price', 'sales')

In [20]:
eda.missing_values_table(merged_df_fe2)

The selected dataframe has 30 columns and 14 columns with missing values.


,Missing Values,% of Total Values
normalized_log_avg_price_5_days,180562,100.0
normalized_log_avg_sales_5_days,180562,100.0
normalized_log_avg_price_10_days,180562,100.0
normalized_log_avg_sales_10_days,180562,100.0
normalized_log_avg_price_15_days,180562,100.0
normalized_log_avg_sales_15_days,180562,100.0
normalized_std_price_15_days,180429,99.9
normalized_std_sales_15_days,180429,99.9
normalized_std_price_10_days,174313,96.5
normalized_std_sales_10_days,174313,96.5


In [21]:
# drop columns with 100% missing values
merged_df_fe2.dropna(axis=1, how='all', inplace=True)


In [22]:
eda.missing_values_table(merged_df_fe2)

The selected dataframe has 24 columns and 8 columns with missing values.


,Missing Values,% of Total Values
normalized_std_price_15_days,180429,99.9
normalized_std_sales_15_days,180429,99.9
normalized_std_price_10_days,174313,96.5
normalized_std_sales_10_days,174313,96.5
normalized_std_price_5_days,110106,61.0
normalized_std_sales_5_days,110106,61.0
brand,142,0.1
name,12,0.0


In [23]:
normalized_cols = [col for col in merged_df_fe2.columns if 'normalized_' in col]
merged_df_fe2[normalized_cols] = merged_df_fe2[normalized_cols].fillna(method='ffill').fillna(method='bfill')


In [24]:
eda.missing_values_table(merged_df_fe2)

The selected dataframe has 24 columns and 2 columns with missing values.


,Missing Values,% of Total Values
brand,142,0.1
name,12,0.0


In [25]:
merged_df_fe3, insuficient_data = fep.filter_stability_periods(merged_df_fe2, 7, 0.04, sku_column='sku', date_column='created_at', price_column='sale_price')

In [26]:
eda.missing_values_table(merged_df_fe3)

The selected dataframe has 24 columns and 2 columns with missing values.


,Missing Values,% of Total Values
brand,122,0.1
name,11,0.0


In [27]:
merged_df_fe3.head()

,sku,status,created_at,sale_price,gender,num_of_item,cost,category,name,brand,department,sales,f_corr_-5.0_1.0,f_corr_-3.0_1.0,f_corr_-2.0_1.0,f_corr_1.0_1.0,f_corr_-1.0_0.5,f_corr_-1.0_0.33,normalized_std_price_5_days,normalized_std_sales_5_days,normalized_std_price_10_days,normalized_std_sales_10_days,normalized_std_price_15_days,normalized_std_sales_15_days
169130,00003E3B9E5336685200AE85D21B4F5E,Shipped,2022-05-14 21:24:59+00:00,99.000000,F,2,56.232000,Pants & Capris,Anne Klein Women's Crop Pant,Anne Klein,Women,4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
169131,00003E3B9E5336685200AE85D21B4F5E,Complete,2022-12-09 03:58:35+00:00,99.000000,F,1,56.232000,Pants & Capris,Anne Klein Women's Crop Pant,Anne Klein,Women,4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
169132,00003E3B9E5336685200AE85D21B4F5E,Complete,2023-05-12 23:08:53+00:00,99.000000,F,1,56.232000,Pants & Capris,Anne Klein Women's Crop Pant,Anne Klein,Women,4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
171824,0004D0B59E19461FF126E3A08A814C33,Processing,2020-08-31 08:42:06+00:00,79.949997,F,1,37.656449,Fashion Hoodies & Sweatshirts,The Bradford Exchange Breast Cancer Support Wo...,Bradford Exchange,Women,6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
171823,0004D0B59E19461FF126E3A08A814C33,Shipped,2022-10-19 10:38:15+00:00,79.949997,F,1,37.656449,Fashion Hoodies & Sweatshirts,The Bradford Exchange Breast Cancer Support Wo...,Bradford Exchange,Women,6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [28]:
## embeddings
from src.data.embeddings import EmbeddingModel
import torch
from sklearn.preprocessing import LabelEncoder
#merged_df_embeddings = merged_df_fe3.copy()


label_encoders = {}
for col in ['category', 'name', 'brand', 'department']:
    le = LabelEncoder()
    merged_df_fe3[col] = le.fit_transform(merged_df_fe3[col])
    label_encoders[col] = le

embedding_dims = [(merged_df_fe3[col].max() + 1, int((merged_df_fe3[col].max() + 1) ** 0.5)) for col in ['category', 'name', 'brand', 'department']]
model = EmbeddingModel(embedding_dims)




#preapre data for embeddings
cat_data = merged_df_fe3[['category', 'name', 'brand', 'department']].values
cat_data_tensor = torch.tensor(cat_data, dtype=torch.long)


In [29]:
import torch.nn as nn
sales_data = merged_df_fe3['sales'].values
sales_data_tensor = torch.tensor(sales_data, dtype=torch.float32).view(-1, 1)

import torch.optim as optim

# Loss and optimizer
criterion = nn.MSELoss()  # Mean Squared Error for a regression problem
optimizer = optim.Adam(model.parameters(), lr=0.01)
epochs = 10  # Number of training epochs

for epoch in range(epochs):
    # Forward pass
    outputs = model(cat_data_tensor)
    
    # Compute loss
    loss = criterion(outputs, sales_data_tensor)

    # Zero gradients, backward pass, optimizer step
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    print(f'Epoch [{epoch+1}/{epochs}], Loss: {loss.item():.4f}')


Inside Forward
Epoch [1/10], Loss: 191.0462
Inside Forward
Epoch [2/10], Loss: 190.7918
Inside Forward
Epoch [3/10], Loss: 190.5376
Inside Forward
Epoch [4/10], Loss: 190.2836
Inside Forward
Epoch [5/10], Loss: 190.0298
Inside Forward
Epoch [6/10], Loss: 189.7762
Inside Forward
Epoch [7/10], Loss: 189.5228
Inside Forward
Epoch [8/10], Loss: 189.2696
Inside Forward
Epoch [9/10], Loss: 189.0167
Inside Forward
Epoch [10/10], Loss: 188.7640


In [30]:

with torch.no_grad():
    embeddings = model.forward(cat_data_tensor).numpy()


Inside Forward


In [31]:
eda.missing_values_table(merged_df_fe3)

The selected dataframe has 24 columns and 0 columns with missing values.


,Missing Values,% of Total Values


In [39]:
# Reset Index Before Concatenating
merged_df_fe3.reset_index(drop=True, inplace=True)
embeddings_df = pd.DataFrame(embeddings)
embeddings_df.reset_index(drop=True, inplace=True)
merged_df_embeddings = pd.concat([merged_df_fe3, embeddings_df], axis=1)


In [40]:
# check lenghts
assert len(merged_df_fe3) == len(embeddings_df), "Dataframes have different lengths."


In [41]:
eda.missing_values_table(merged_df_embeddings)

The selected dataframe has 247 columns and 0 columns with missing values.


,Missing Values,% of Total Values


In [33]:
#merged_df_embeddings = pd.concat([merged_df_fe3, pd.DataFrame(embeddings)], axis=1)
#merged_df_embeddings.drop(['category', 'name', 'brand', 'department'], axis=1, inplace=True)


In [45]:
import numpy as np
# Create a correlation matrix to check for multicollinearity
corr_matrix = merged_df_embeddings.corr().abs()
# Select upper triangle of correlation matrix
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool))
# Find index of feature columns with correlation greater than 0.95
to_drop = [column for column in upper.columns if any(upper[column] > 0.95)]
print(f"Features to drop: {to_drop}")

Features to drop: ['cost', 'f_corr_-3.0_1.0', 'f_corr_-2.0_1.0', 'f_corr_1.0_1.0', 'f_corr_-1.0_0.5', 'f_corr_-1.0_0.33', 222]


In [48]:
from statsmodels.stats.outliers_influence import variance_inflation_factor
import pandas as pd

# Drop non-numeric columns for VIF calculation
numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
new_df = merged_df_embeddings.select_dtypes(include=numerics)


# Calculate VIF for each feature
vif_data = pd.DataFrame()
vif_data["feature"] = new_df.columns

vif_data["VIF"] = [variance_inflation_factor(new_df.values, i) for i in range(len(new_df.columns))]

# Identify features with a high VIF value
high_vif_features = vif_data[vif_data["VIF"] > 10]["feature"].tolist()

print(f"Features with high VIF should be removed or examined: {high_vif_features}")



Features with high VIF should be removed or examined: ['sale_price', 'cost', 'department', 'f_corr_-5.0_1.0', 'f_corr_-3.0_1.0', 'f_corr_-2.0_1.0', 'f_corr_1.0_1.0', 'f_corr_-1.0_0.5', 'f_corr_-1.0_0.33', 222]


In [51]:
print(to_drop)
print(high_vif_features)

['cost', 'f_corr_-3.0_1.0', 'f_corr_-2.0_1.0', 'f_corr_1.0_1.0', 'f_corr_-1.0_0.5', 'f_corr_-1.0_0.33', 222]
['sale_price', 'cost', 'department', 'f_corr_-5.0_1.0', 'f_corr_-3.0_1.0', 'f_corr_-2.0_1.0', 'f_corr_1.0_1.0', 'f_corr_-1.0_0.5', 'f_corr_-1.0_0.33', 222]


In [54]:
train_dataset = merged_df_embeddings.drop(columns=['f_corr_-3.0_1.0', 'f_corr_-2.0_1.0', 'f_corr_1.0_1.0', 'f_corr_-1.0_0.5', 'f_corr_-1.0_0.33', 222], axis=1)

In [56]:
# drop categorical features used in embeddings

train_dataset = train_dataset.drop(columns=['category', 'name', 'brand', 'department'], axis=1)

In [59]:
train_dataset  =  fep.datetime_transform(train_dataset,'created_at', ['day_of_the_year'])


In [63]:
train_dataset.status.unique()
# drop skus with status "Returned" and "Cancelled"
train_dataset = train_dataset.query('status != "Returned" and status != "Cancelled"')


In [65]:
# use label encoder for status and gender


train_dataset['status'] = le.fit_transform(train_dataset['status'])
train_dataset['gender'] = le.fit_transform(train_dataset['gender'])



In [68]:
train_dataset.sort_values(by=['created_at'], inplace=True)

In [69]:
# print first and last date
print(train_dataset.created_at.min())
print(train_dataset.created_at.max())


2019-01-11 13:05:59+00:00
2023-09-11 18:25:47.230646+00:00


In [71]:
from typing import Dict, Union
import pandas as pd

def describe_time_series(df: pd.DataFrame, date_column: str, sales_column: str) -> Dict[str, Union[int, Dict[int, Dict[str, int]]]]:
    """
    Function to provide a comprehensive description of the time series data.
    
    Args:
    - df (pd.DataFrame): DataFrame containing the time series data.
    - date_column (str): Column name containing the date information.
    
    Returns:
    - Dict: Description containing number of years, months per year, and average sales per month.
    """

    # Convert date column to datetime if not already
    df[date_column] = pd.to_datetime(df[date_column])

    # Extract year and month from date
    df['year'] = df[date_column].dt.year
    df['month'] = df[date_column].dt.month

    # Initialize results dictionary
    result = {
        "total_years": 0,
        "years_detail": {}
    }

    # Total number of unique years
    total_years = df['year'].nunique()
    result["total_years"] = total_years

    # Iterate over each year to find details
    for year in df['year'].unique():
        year_data = df[df['year'] == year]
        
        # Number of unique months in this year
        unique_months = year_data['month'].nunique()
        
        # Average sales per month
        avg_sales_per_month = year_data.groupby('month')[sales_column].mean().to_dict()

        # Add details to result
        result['years_detail'][year] = {
            "unique_months": unique_months,
            "avg_sales_per_month": avg_sales_per_month
        }

    return result

# Example usage (assuming 'merged_df' contains 'date' and 'sales' columns)
# describe_time_series(merged_df, 'date')






In [72]:
describe_time_series(train_dataset, 'created_at', 'sales')

{'total_years': 5,
 'years_detail': {2019: {'unique_months': 12,
   'avg_sales_per_month': {1: 12.666666666666666,
    2: 13.74,
    3: 13.682926829268293,
    4: 13.419117647058824,
    5: 13.859903381642512,
    6: 13.095959595959595,
    7: 14.178807947019868,
    8: 13.647058823529411,
    9: 13.664021164021165,
    10: 13.160401002506266,
    11: 13.606490872210953,
    12: 13.599616858237548}},
  2020: {'unique_months': 12,
   'avg_sales_per_month': {1: 13.807692307692308,
    2: 13.877697841726619,
    3: 13.811475409836065,
    4: 13.593667546174142,
    5: 13.701594533029613,
    6: 13.743362831858407,
    7: 13.716026241799439,
    8: 13.991035856573705,
    9: 13.58987090367428,
    10: 13.746280991735537,
    11: 14.052842273819055,
    12: 13.537164750957855}},
  2021: {'unique_months': 12,
   'avg_sales_per_month': {1: 13.979452054794521,
    2: 13.53061224489796,
    3: 13.515333333333333,
    4: 13.603861517976032,
    5: 13.754950495049505,
    6: 13.868031854379977,
 

In [86]:
# print min and max date
print(train_dataset.created_at.min())
print(train_dataset.created_at.max())

2019-01-11 13:05:59+00:00
2023-09-11 18:25:47.230646+00:00


In [94]:
# First localize to UTC and then convert to naive datetime
train_dataset['created_at'] = pd.to_datetime(train_dataset['created_at'], utc=True).dt.tz_localize(None)

# Check unique dates and data types after conversion
print("Unique dates after conversion:", train_dataset['created_at'].unique())
print("Data type after conversion:", train_dataset['created_at'].dtype)



Unique dates after conversion: ['2019-01-11T13:05:59.000000000' '2019-01-17T07:00:33.000000000'
 '2019-01-18T13:39:38.000000000' ... '2023-09-11T17:30:37.049453000'
 '2023-09-11T18:23:00.836955000' '2023-09-11T18:25:47.230646000']
Data type after conversion: datetime64[ns]


In [110]:

from src.data.data_splitter import DataSplitter


data_splitter = DataSplitter(train_dataset, 'sales','sku', 'created_at', 36, 6, 4)
X_train, y_train, X_val, y_val, X_test, y_test = data_splitter.split_data()

Debug: Data type of date column before processing: datetime64[ns]
Debug: Unique dates before processing: ['2019-01-11T13:05:59.000000000' '2019-01-17T07:00:33.000000000'
 '2019-01-18T13:39:38.000000000' ... '2023-09-11T17:30:37.049453000'
 '2023-09-11T18:23:00.836955000' '2023-09-11T18:25:47.230646000']
Number of SKUs in train: 4386
Number of SKUs in validation: 28576
Number of SKUs in test: 5317
Min date in dataset: 2019-01-11 13:05:59
Max date in dataset: 2023-09-11 18:25:47.230646
Train-Val threshold: 2020-03-11 18:25:47.230646
Val-Test threshold: 2023-08-14 18:25:47.230646
Training data covers from 2019-01-21 10:37:26 to 2020-03-11 15:46:38
Validation data covers from 2020-03-12 22:31:23 to 2023-08-14 07:48:06
Test data covers from 2023-08-14 18:56:53 to 2023-09-11 17:30:37.049453


In [111]:
print('X_val info', X_val.info())
print('X_train info', X_train.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1294 entries, 83409 to 78374
Columns: 238 entries, sku to month
dtypes: datetime64[ns](1), float32(221), float64(9), int64(6), object(1)
memory usage: 1.3+ MB
X_val info None
<class 'pandas.core.frame.DataFrame'>
Int64Index: 546 entries, 48710 to 13669
Columns: 238 entries, sku to month
dtypes: datetime64[ns](1), float32(221), float64(9), int64(6), object(1)
memory usage: 548.1+ KB
X_train info None


In [114]:
X_train.head()

,sku,status,sale_price,gender,num_of_item,cost,f_corr_-5.0_1.0,normalized_std_price_5_days,normalized_std_sales_5_days,normalized_std_price_10_days,normalized_std_sales_10_days,normalized_std_price_15_days,normalized_std_sales_15_days,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,created_at_day_of_the_year,year,month
48710,5421E013565F7F1AFA0CFE8AD87A99AB,1,46.990002,0,1,23.166071,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.618097,0.213610,0.194460,-0.736318,-0.128905,1.207151,0.485925,-0.050880,-0.197749,-0.057977,0.582514,-0.033598,1.127005,0.732027,0.538784,0.508141,-0.818943,0.273989,-0.959761,-0.617215,-0.218731,-0.291490,0.037319,0.121658,1.289786,1.088777,0.154329,-0.983612,-1.703685,0.249646,1.382619,-1.325768,-1.374904,-2.372104,0.187922,1.496713,1970-01-01 00:00:00.000000000,1.201915,0.369966,-1.010207,0.301286,-0.050044,-0.066550,-0.162605,-0.553055,1.202927,0.839608,0.281543,-0.575626,-1.182218,-0.199428,2.012374,1.175463,-0.887381,0.863653,1.353978,0.945790,1.379825,-1.020125,-0.244657,0.716591,-0.214237,1.455720,0.539852,0.447123,-1.568236,-0.790396,-0.108470,0.330332,-0.019592,0.096214,-0.652981,0.612672,-1.184975,-0.038366,-0.363132,-0.445671,0.089539,-0.965762,-1.465110,0.821124,-0.233077,-0.980053,-1.907379,1.748868,-0.452622,0.306807,0.118778,0.293713,0.990599,-0.247371,0.651318,1.713531,0.526332,0.898758,0.943439,-0.714881,-1.414468,0.508031,2.038816,0.885909,2.014792,0.680260,-1.312942,1.203345,0.319201,-0.622913,-1.187147,-0.891090,1.049363,0.745267,-0.762985,-0.029543,0.247401,-0.996835,-0.300898,-0.656780,1.043540,-0.368252,-1.774425,-1.173386,-1.054245,-0.028838,0.482536,0.862234,0.385652,-2.345551,-3.109762,0.848012,-0.044475,0.160467,0.226374,0.891752,1.507178,1.777077,-0.616642,1.766401,-1.213559,-0.319520,1.015720,1.095911,0.915277,0.687072,0.608772,0.496810,-0.149331,-1.915155,0.136365,-0.785196,1.051046,-0.254127,-0.576281,0.420114,-1.041021,3.093266,-0.500465,-0.345949,0.397736,1.233427,-0.949307,1.262350,-1.515631,0.296013,0.038116,0.309216,-0.717660,0.394764,-0.322131,-0.197474,-0.178135,0.396525,1.689622,0.135936,2.463261,-0.288633,0.984552,0.557286,-1.179625,1.468262,0.125309,-0.693422,0.348455,0.969624,1.154254,-0.863502,0.782441,0.436930,1.423537,-0.213930,1.065327,0.028801,-0.215653,0.795972,-1.134716,0.506972,0.533417,1.566646,2.094918,-1.778114,-0.217691,1.015229,-0.849052,0.081957,-0.894382,-0.508788,-1.747872,0.571601,1.535555,0.923321,0.029534,0.515496,-0.366679,0.054093,0.237928,-0.630707,-1.181410,0.005756,-1.001396,-0.038317,0.241639,-0.263245,-1.197127,21,2019,1
111440,BF1B2F4B901C21A1D8645018EA9AEB05,1,21.990000,0,1,11.742660,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.278280,0.016417,-0.460472,0.332568,-0.213493,-0.833412,-0.719302,-0.886362,0.262113,0.513147,0.610317,-0.405723,1.536282,-0.612963,1.484782,-0.877877,-0.880291,-0.691230,0.669534,1.423011,-1.320920,-0.596399,-0.108770,-1.363332,1.400030,0.569100,-0.610503,-0.393085,-0.249465,1.715142,-2.020521,-1.031594,-1.367871,-0.543092,-2.677600,-0.845990,1969-12-31 23:59:59.999999999,-0.471224,2.338659,0.060505,-0.694462,0.195182,-0.707157,0.360006,0.232534,-0.527404,1.643142,-1.105963,0.753707,-1.051327,1.549529,0.757793,1.244341,-0.697667,-1.268019,1.184420,-0.398877,-0.739393,-0.278463,1.003109,-1.019797,-0.535316,0.255902,-0.438590

In [115]:
# drop 'sku', 'year', 'month' from X_train and X_val
X_train.drop(columns=['sku', 'year', 'month'], inplace=True)
X_val.drop(columns=['sku', 'year', 'month'], inplace=True)

X_test.drop(columns=['year', 'month'], inplace=True)

In [121]:
# drop column "36" from X_train and X_val
X_train.drop(columns=[36], inplace=True)
X_val.drop(columns=[36], inplace=True)
X_test.drop(columns=[36], inplace=True)


In [126]:
from src.train.quantile_training import DemandCurveTrainer
trainer = DemandCurveTrainer(X_train, y_train, X_val, y_val,  tune_params=True)
trainer.train_models()
trainer.evaluate_models()
#trainer.plot_feature_importance()
# Predict demand curve (Assume test_data is prepared with SKUs and prices)
sku_demand_curves, demand_curve_pricing, price_points_per_sku = trainer.predict_demand_curve(X_test, 'sku', 'sale_price')


[I 2023-09-11 18:53:15,461] A new study created in memory with name: no-name-03ebe45f-6589-4bbd-82f5-6c0be9e54914


[LightGBM] [Warning] min_data_in_leaf is set=9, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=9


[I 2023-09-11 18:53:15,804] Trial 0 finished with value: 0.7046730446906897 and parameters: {'max_depth': 5, 'min_data_in_leaf': 9, 'subsample': 0.88547910669042, 'n_estimators': 58, 'learning_rate': 0.15395121530210276, 'colsample_bytree': 0.6470038017577991}. Best is trial 0 with value: 0.7046730446906897.


[LightGBM] [Warning] min_data_in_leaf is set=14, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=14


[I 2023-09-11 18:53:16,223] Trial 1 finished with value: 9.258408050989638 and parameters: {'max_depth': 7, 'min_data_in_leaf': 14, 'subsample': 0.7738623467267365, 'n_estimators': 78, 'learning_rate': 0.012354244383720132, 'colsample_bytree': 0.6848661167316181}. Best is trial 0 with value: 0.7046730446906897.


...

[LightGBM] [Warning] min_data_in_leaf is set=5, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=5


[I 2023-09-11 18:53:34,471] Trial 48 finished with value: 0.4932330997907338 and parameters: {'max_depth': 5, 'min_data_in_leaf': 5, 'subsample': 0.9447739946268753, 'n_estimators': 89, 'learning_rate': 0.18919308556045694, 'colsample_bytree': 0.7249842733097855}. Best is trial 42 with value: 0.43660756312979615.


[LightGBM] [Warning] min_data_in_leaf is set=8, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=8


[I 2023-09-11 18:53:34,889] Trial 49 finished with value: 0.6869216366887604 and parameters: {'max_depth': 6, 'min_data_in_leaf': 8, 'subsample': 0.8523345105518135, 'n_estimators': 70, 'learning_rate': 0.1911622130765715, 'colsample_bytree': 0.8111344472912768}. Best is trial 42 with value: 0.43660756312979615.


Fitting model for quantile 0.1
[LightGBM] [Warning] min_data_in_leaf is set=10, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=10
[1]	valid_0's quantile: 0.519981
[2]	valid_0's quantile: 0.509422
[3]	valid_0's quantile: 0.499093
[4]	valid_0's quantile: 0.492477
[5]	valid_0's quantile: 0.484207
[6]	valid_0's quantile: 0.480057
[7]	valid_0's quantile: 0.473872
[8]	valid_0's quantile: 0.467611
[9]	valid_0's quantile: 0.463298
[10]	valid_0's quantile: 0.458618
[11]	valid_0's quantile: 0.452573
[12]	valid_0's quantile: 0.449456
[13]	valid_0's quantile: 0.445496
[14]	valid_0's quantile: 0.442669
[15]	valid_0's quantile: 0.438586
[16]	valid_0's quantile: 0.436881
[17]	valid_0's quantile: 0.43393
[18]	valid_0's quantile: 0.431137
[19]	valid_0's quantile: 0.428422
[20]	valid_0's quantile: 0.425812
[21]	valid_0's quantile: 0.423952
[22]	valid_0's quantile: 0.422991
[23]	valid_0's quantile: 0.420801
[24]	valid_0's quantile: 0.419148
[25]	valid_0's quantile: 0.418232
[26]	va

In [131]:
 demand_curve_pricing

{'0129A222FE10C9896DB191C537694699': {0.1: [(29.78400077819824,
    6.884668001316594),
   (31.53600082397461, 6.884668001316594),
   (33.288000869750974, 6.884668001316594),
   (34.3392008972168, 6.884668001316594),
   (35.040000915527344, 6.884668001316594),
   (35.74080093383789, 6.884668001316594),
   (36.792000961303714, 6.884668001316594),
   (38.544001007080084, 6.884668001316594),
   (40.29600105285644, 6.884668001316594),
   (42.04800109863281, 6.884668001316594)],
  0.5: [(29.78400077819824, 8.70770057534645),
   (31.53600082397461, 8.70770057534645),
   (33.288000869750974, 8.70770057534645),
   (34.3392008972168, 8.70770057534645),
   (35.040000915527344, 8.500590005119307),
   (35.74080093383789, 8.500590005119307),
   (36.792000961303714, 8.500590005119307),
   (38.544001007080084, 8.500590005119307),
   (40.29600105285644, 8.707399830019309),
   (42.04800109863281, 8.707399830019309)],
  0.9: [(29.78400077819824, 13.516789693105997),
   (31.53600082397461, 13.51678969310

In [127]:
import plotly.express as px
import pandas as pd


def plot_demand_curve_for_sku(sku_demand_curves, price_points_per_sku, selected_sku):
    # Create the DataFrame properly
    data = []
    for sku, sku_demand_curve in sku_demand_curves.items():
        if sku == selected_sku:  # Only proceed if the SKU matches
            for q, demand_predictions in sku_demand_curve.items():
                for price, prediction in zip(price_points_per_sku[sku], demand_predictions):
                    data.append({'SKU': sku, 'Price Points': price, 'Predicted Sales': prediction, 'Quantile': f'Quantile {q}'})
    
    df = pd.DataFrame(data)
    
    # Create the Plotly figure for the selected SKU
    fig = px.line(df, x='Price Points', y='Predicted Sales', color='Quantile',
                  title=f'Demand Curve for SKU: {selected_sku}',
                  labels={'Price Points': 'Price Points', 'Predicted Sales': 'Predicted Sales'})
    fig.show()

In [132]:
plot_demand_curve_for_sku(sku_demand_curves, price_points_per_sku, selected_sku="0129A222FE10C9896DB191C537694699")